In [7]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
from os.path import exists, realpath
from urllib.parse import quote
import json
import glob
import csv, glob, requests, pdfplumber, io 
import unicodedata
current_record = 0
current_set = 0
fails = 0

In [8]:
s = requests.Session()
cookie = 'PHPSESSID=c8hcufru9lt9puq06ct3map455'
def get_case_deets(cino, case_no, court_no, state_code, dist_code):
    
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': cookie
    }

    data = {
      '__csrf_magic': 'sid:22e25a4a3b8a2c3dd3a08f8c43e207354a236ed2,1641670946',
      'state_code': state_code,
      'dist_code': dist_code,
      'cino': cino,
      'case_no': case_no,
      'appFlag': '',
      'court_code': court_no
    }

    response = s.post('https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/o_civil_case_history.php', headers=headers,data=data)
    return (response.content, 0)

def get_case_document(link):

    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': cookie

    }
    response = s.get(link, headers=headers)
    return (response)

def process(soup, cino):
    deets = get_all(soup)
    if len(deets[1])<1:
        return 0
    disposed_date = " "
    disposed_reason = " "
    disposed_year = " "
    filing_number= deets[0][6][15:25].split('/')[0]
    filing_date = deets[0][8][-10:]
    filing_year = filing_date[-4:]
    registration_number = deets[0][9][21:31].split('/')[0]
    registration_date = deets[0][11][19:]
    registration_year = registration_date[-4:]
    stage = deets[0][16].split(": ")[-1].strip()     
    stage = re.sub(r'[^a-zA-Z ]', '',stage)  
    
    if 'CASE DISPOSED' in stage.upper():
        disposed_date = deets[0][15].split(":")[-1].strip()
        disposed_year = disposed_date[-4:]
        disposed_reason =deets[0][17].split(":")[-1].strip()
        
        judge = deets[0][18].split(':')[-1].strip()[4:].title()
        bench = deets[0][19].split(':')[-1].strip().title()
    else:
        judge = deets[0][17].split(':')[-1].strip()[4:].title()
        bench = deets[0][18].split(':')[-1].strip().title()
    if bench == '':
        bench = ' '
    statute = deets[1][0]
    provision = deets[1][1]
    if statute == 'History of Case Hearing':
        statute = " "
        provision = " "
    try:
        first_hearing = deets[0][14].split(": ")[1]
    except IndexError:
        first_hearing = ""

    category, sub_category, sub_sub_category, *_ = get_category_details(deets[-1])

    
    party_subview = [x for x in deets[0] if x[:2]=='1)']
    #print(party_subview)
    p_advocate, p_length, p1, p2, p3, p4, *_ = clean_parties(party_subview[0])
    r_advocate, r_length, r1, r2, r3, r4, *_ = clean_parties(party_subview[1])

    

    try:
        last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
        last_hearing_date = last_hearing[-3].get_text()
        last_hearing_matter = last_hearing[-1].get_text()
    except IndexError:
        last_hearing_date = ""
        last_hearing_matter = ""
    
    try:
        order_link = soup.select("#secondpage > div:nth-child(13) > table.order_table > tbody > tr:nth-child(2) > td:nth-child(4) > a")['href']
    except:
        order_link = " "
    try:
        acts = soup.select("table#act_table tr")[1:]
        provisions = ""
        for act in acts:
            
            title = act.select("td")[0].get_text()
            sections = act.select("td")[1].get_text()
            if "-" in sections:
                continue
            else:
                sections = sections.replace(",", " ")
                string = f"{title}: {sections}; "
                provisions+=string

    except IndexError as e:
        provisions = "N/A"
        raise e

    FIR_details = soup.select("span.FIR_details_table label")
    try:    
        if FIR_details:
            station = FIR_details[0].get_text()
            fir_number = FIR_details[1].get_text()
            fir_year = FIR_details[2].get_text()
        else:
            station, fir_number, fir_year = ("N/A","N/A","N/A")
    except IndexError as e:
        
        station, fir_number, fir_year = ("N/A","N/A","N/A")

    try:
        case_type = soup.select(".case_details_table")[0].get_text()
        case_type = case_type.split(":")[-1].strip()
    except IndexError as e:
        case_type = ""
        raise e
    
    if p_length > 1:
        p1 += " & Ors."
    if r_length > 1:
        r1 += " & Ors."
    rows = [p1,r1, str(p_length), str(r_length), cino, case_type, filing_number, filing_year, filing_date, registration_number, registration_year, registration_date, fir_number, station, fir_year, provisions, stage, disposed_date, disposed_year, disposed_reason,]
    rows = [x.replace(":\xa0", "") for x in rows]
    #print(rows)
    return rows

def print_all(soup):
    import unicodedata
    f = open('log.txt', 'a')
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    f.write(f"Record: {current_record}: ")
    f.write(str([normal(x) for x in soup.find_all('span')]))
    f.write('td\n')
    f.write(str([normal(x) for x in soup.find_all('td')]))
    f.write('tr\n')
    f.write(str([normal(x) for x in soup.find_all('tr')]))
    f.write('\n')
    last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
    f.write(str(last_hearing))
    f.write('\n\n')

    f.close()
def get_all(soup):
    import unicodedata
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    return (([normal(x) for x in soup.find_all('span')]), [normal(x) for x in soup.find_all('td')], [normal(x) for x in soup.find_all('tr')])\



def get_category_details(items):
    temp_category = [x.split('\n')[-1] for x in items if "Category\n" in x]
    while len(temp_category)<3:
        temp_category.append(" ")
    return temp_category

def clean_parties(parties):
    parties = re.split(r'\d+\)', parties)
    clean_parties = []
    advocates = []
    for ind, party in enumerate(parties):
        if "Advocate" in party:
            advocate = party.split("Advocate")[-1]
            advocate = re.sub(r'[^a-zA-Z ]', '',advocate).strip()
            if "FOR R" in advocate:
                advocate = advocate.split("FOR")[0]
            #print(advocate)
            advocates.append(advocate)
        
        temp_party = party.split("Advocate")[0].strip()
        if temp_party:
            temp_party = re.sub(r'[^a-zA-Z \\ \)\(]', '',temp_party)
            clean_parties.append(temp_party)
           
    if not advocates:
        advocate = " "
    else:
        advocate = advocates[0]

    clean_parties.insert(0, len(clean_parties))
    clean_parties.insert(0, advocate)
    
    while len(clean_parties) < 6:
        clean_parties.append(" ")

    return clean_parties



In [18]:
import glob
all_files = glob.glob("saloni/case_list/*.txt")
all_cases = []
for file in all_files:
    with open(file, 'r') as f:
        all_cases = []
        if '_Disp' in file:
            disp="disposed"
        else:
            disp="pending"

        x = json.loads(f.read())
        for cases, court_code, court in zip(x['con'], x['court_code'], x['courtNameArr']):
            if(cases and court_code and court):
                
                all_cases.append({
                'court_no': court_code,
                'cases': json.loads(cases),
                'court': court,
                'state_code': x['stateCode'],
                'dist_code': x['distCode'],
                'disp': disp
            })


count = 0
for cases in all_cases:
    count+=len(cases['cases'])

print(f"Processing {count} cases total")


26 6 1
saloni/case_list/376_1,2,3,5_26_6_Disposed.txt
Processing 14 cases total


In [16]:
for x in all_cases:
    for case in x['cases']:
        if case['cino'] == 'DLSW010027652016':
            p = case
            print(x['state_code'], x['dist_code'], x['court_no'])
p

26 6 1


{'case_no': '202304411782016',
 'pet_name': 'STATE',
 'case_no2': 441178,
 'case_year': 2016,
 'res_name': 'VIJENDER SINGH',
 'party_name1': 'N',
 'party_name2': 'N',
 'date_of_decision': '2017-04-03',
 'cino': 'DLSW010027652016',
 'type_name': 'SC',
 'hideparty': 'N'}

In [6]:
for set_no in range(current_set, len(all_cases)):

    court_code = all_cases[current_set]['court_no']
    cases = all_cases[current_set]['cases']
    court = all_cases[current_set]['court']
    state_code = all_cases[current_set]['state_code']
    dist_code = all_cases[current_set]['dist_code']
    disp = all_cases[current_set]['disp']
    print(f"Starting {court}, {len(cases)} pending, {fails} failed so far. Set {current_set} of {len(all_cases)}")

    filename = f"saloni/{court}_{disp}.csv"
    csv_file = open(filename,'w', encoding='utf-8', newline='')
    csvwriter = csv.writer(csv_file)
    if current_record == 0:
        fields = ['Petitioner','Respondent', "Petitioner Count", "Respondent Count", "Case Type", "CINO", 'Filing Number', 'Filing Year', 'Filing Date', "Registration Number", 'Registration Year','Registration Date', "FIR Number", "Station", "FIR Year", "Provisions Charged", 'Stage','Disposed Date', 'Disposed Year', 'Disposal Reason', "Court", "Extract",]
        csvwriter.writerow(fields) 
    
    for x in range(current_record, len(cases)):
        print(f"Processing case {current_record}")
        cino = cases[x]['cino']
        case_no = cases[x]['case_no']
        #print(current_record, cino, case_no)
        for i in range(5):
            try:
                page, downloaded = get_case_deets(cino, case_no, court_code, state_code, dist_code)
                soup = BeautifulSoup(page)
                break
            except Exception as e:
                time.sleep(0.5+i)
                print(e, "Retrying")
        else:
            current_record+=1
            fails+=1  
            continue

        try:
            row = process(soup, cino)
            row.append(court)
            orders = soup.select("table.order_table tr")[1:]
            if len(orders) == 0:
                orders = []
            elif len(orders)<2:
                orders = [orders[-1]]
            else:
                orders = orders[-2:]
            
            for index, order in enumerate(orders):
                details = [x.get_text().strip() for x in order.select('td')]
                if not details:
                    continue
                date = details[-2]
                name = details[-1]
                query = order.select('a')[0]['href'].replace("  ", '%20')
                query = query.replace(" ", "%20")
                if 'judg' not in name.lower():
                    print("NOT JUDGEMENT", name)
                    continue
                link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{query}"
                
                order_name = f"saloni/orders/{row[3]} of {row[4]}_{name.title()}_{cino}_{index}.pdf"
                doc = get_case_document(link)
                if doc.headers['Content-Type'] == 'application/pdf':
                    with pdfplumber.open(io.BytesIO(doc.content)) as pdf:
                        texts = []
                        for page in pdf.pages:
                            text = page.extract_text()
                            texts.append(text)
                        paras = " ".join(texts)
                        paras = paras.replace("\n", " ")
                        line = re.findall("[^.?!]*(?<=[.?\s!])DNA(?=[\s.?!])[^.?!]*[.?!]", paras)
                        if line:
                            print(f"DNA FOUND, {line}")
                            with open(order_name, 'wb') as f:      
                                f.write(doc.content)
                                file_name = f'=hyperlink("{order_name}", {"|".join(line)})'
                                row.extend([name, file_name])

                        
                else:
                    print("No file", doc.headers['Content-Type'])

            if row != 0:
                print(row)
                csvwriter.writerow(row)
        except Exception as e:      
            fails+=1    
            with open(f"{current_record}.html", 'w', encoding='utf-8') as f:
                f.write(str(soup))
            raise(e)

        current_record+=1
    print(f"Processed {current_record} cases")
    csv_file.close()
    current_record = 0
    current_set = current_set+1
    

Starting District and Sessions Judge, South , Saket, 380 pending, 0 failed so far. Set 0 of 72
Processing case 0
['SAURAB AND ORS', 'STATE', '1', '1', 'DLST010072892019', 'CA - CRIMINAL APPEAL', '4174', '2019', '22-10-2019', '392', '2019', '23-10-2019', 'N/A', 'N/A', 'N/A', 'IPC: 302 34; ', 'Case disposed', '09th December 2019', '2019', 'Contested--APPEAL ACCEPTED', 'District and Sessions Judge, South , Saket']
Processing case 1
['ABHISHEK', 'STATE', '1', '1', 'DLST010055962019', 'CA - CRIMINAL APPEAL', '3239', '2019', '20-08-2019', '323', '2019', '29-08-2019', 'N/A', 'N/A', 'N/A', 'IPC: 302 ; ', 'Case disposed', '21st October 2019', '2019', 'Contested--ALLOWED', 'District and Sessions Judge, South , Saket']
Processing case 2


KeyboardInterrupt: 

In [13]:
p

{'case_no': '202304411782016',
 'pet_name': 'STATE',
 'case_no2': 441178,
 'case_year': 2016,
 'res_name': 'VIJENDER SINGH',
 'party_name1': 'N',
 'party_name2': 'N',
 'date_of_decision': '2017-04-03',
 'cino': 'DLSW010027652016',
 'type_name': 'SC',
 'hideparty': 'N'}

In [17]:
print(f"Processing case {current_record}")
cino = p['cino']
case_no = p['case_no']
#print(current_record, cino, case_no)
for i in range(5):
    try:
        page, downloaded = get_case_deets(cino, case_no, 1, 26, 6)
        soup = BeautifulSoup(page)
        break
    except Exception as e:
        time.sleep(0.5+i)
        print(e, "Retrying")
else:
    current_record+=1
    fails+=1  
    

try:
    row = process(soup, cino)
    row.append(court)
    orders = soup.select("table.order_table tr")[1:]
    if len(orders) == 0:
        orders = []
    elif len(orders)<2:
        orders = [orders[-1]]
    else:
        orders = orders[-2:]
    
    for index, order in enumerate(orders):
        details = [x.get_text().strip() for x in order.select('td')]
        if not details:
            continue
        date = details[-2]
        name = details[-1]
        query = order.select('a')[0]['href'].replace("  ", '%20')
        query = query.replace(" ", "%20")
        if 'judg' not in name.lower():
            print("NOT JUDGEMENT", name)
            continue
        link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{query}"
        
        order_name = f"saloni/orders/{row[3]} of {row[4]}_{name.title()}_{cino}_{index}.pdf"
        doc = get_case_document(link)
        if doc.headers['Content-Type'] == 'application/pdf':
            with pdfplumber.open(io.BytesIO(doc.content)) as pdf:
                texts = []
                for page in pdf.pages:
                    text = page.extract_text()
                    texts.append(text)
                paras = " ".join(texts)
                paras = paras.replace("\n", " ")
                line = re.findall("[^.?!]*(?<=[.?\s!])DNA(?=[\s.?!])[^.?!]*[.?!]", paras)
                if line:
                    print(f"DNA FOUND, {line}")
                    with open(order_name, 'wb') as f:      
                        f.write(doc.content)
                        file_name = f'=hyperlink("{order_name}", {"|".join(line)})'
                        row.extend([name, file_name])

                
        else:
            print("No file", doc.headers['Content-Type'])

    if row != 0:
        print(row)
        csvwriter.writerow(row)
except Exception as e:      
    fails+=1    
    with open(f"{current_record}.html", 'w', encoding='utf-8') as f:
        f.write(str(soup))
    raise(e)


Processing case 1
NOT JUDGEMENT COPY OF ORDER
DNA FOUND, ['\xa0No\xa0male\xa0DNA\xa0was\xa0isolated\xa0from\xa0exhibits\xa0pertaining\xa0to the\xa0prosecutrix.']
['STATE & Ors.', 'VIJENDER SINGH', '2', '1', 'DLSW010027652016', 'SC - SESSIONS CASE', '9984', '2016', '20-05-2016', '441178', '2016', '20-05-2016', '98', 'Dwarka Sector-23', '2016', 'Indian Penal Code: 376 506 328; ', 'Case disposed', '03rd April 2017', '2017', 'Contested--ACQUITTED', 'POLC and POIT, Rouse Avenue, New Delhi', 'COPY OF JUDGMENT', '=hyperlink("saloni/orders/1 of DLSW010027652016_Copy Of Judgment_DLSW010027652016_1.pdf", \xa0No\xa0male\xa0DNA\xa0was\xa0isolated\xa0from\xa0exhibits\xa0pertaining\xa0to the\xa0prosecutrix.)']


NameError: name 'csvwriter' is not defined

In [34]:
row[10]

'Sangam Vihar'

In [ ]:
import glob, csv
processed_cases = []
files = glob.glob("output/*.csv")
if glob.glob(files[0]):
    with open(files[0], 'r') as f:
        csv_reader = csv.reader(f)
        x = list(csv_reader)
        processed_cases.extend([c[2] for c in x])
print(processed_cases)

In [7]:
import glob
all_files = glob.glob("saloni/case_list/*.txt")
all_cases = []
for file in all_files:
    with open(file, 'r') as f:
        if '_Disp' in file:
            disp="disposed"
        else:
            disp="pending"

        x = json.loads(f.read())
        for cases, court_code, court in zip(x['con'], x['court_code'], x['courtNameArr']):
            if(cases and court_code and court):
                all_cases.append({
                'court_no': court_code,
                'cases': json.loads(cases),
                'court': court,
                'state_code': x['stateCode'],
                'dist_code': x['distCode'],
                'disp': disp
            })
count = 0
for cases in all_cases:
    count+=len(cases['cases'])

print(f"Processing {count} cases total")
len(all_cases[0]['cases'])

Processing 31599 cases total


380

In [27]:
current_set=0
fails = 0
current_record = 0
all_cases = [x for x in all_cases if x['disp']=='disposed']

In [42]:
current_record = 715

In [37]:
logs = open('logs.txt', 'a')

In [1]:
import pdfplumber

with pdfplumber.open("saloni/orders/3038 of 2017_Copy Of Judgment_DLSW010074982017_1.pdf") as pdf:
    texts = []
    for page in pdf.pages:
        text = page.extract_text()
        texts.append(text)


FileNotFoundError: [Errno 2] No such file or directory: 'saloni/orders/3038 of 2017_Copy Of Judgment_DLSW010074982017_1.pdf'

In [5]:
import re
with pdfplumber.open("9984 of 2016_Copy Of Judgment_DLSW010027652016_1.pdf") as pdf:
    texts = []
    for page in pdf.pages:
        text = page.extract_text()
        texts.append(text)
    paras = " ".join(texts)
    paras = paras.replace("\n", " ")
    line = re.findall("[^.?!]*(?<=[.?\s!])DNA(?=[\s.?!])[^.?!]*[.?!]", paras)
line


['\xa0No\xa0male\xa0DNA\xa0was\xa0isolated\xa0from\xa0exhibits\xa0pertaining\xa0to the\xa0prosecutrix.']

['2017,   DNA   could   not   be   isolated   from   product   of conception of prosecutrix due to poor preservation of sample.']

In [5]:
import io
from bs4 import BeautifulSoup
from twocaptcha import TwoCaptcha
import json
import requests
import re

def get_court_complex_cases(link, section=379):
    #Get captcha and site
    print("Starting ", link)
    headers = {
    'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Referer': 'https://services.ecourts.gov.in/',
    'Sec-Fetch-Dest': 'image',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}
    s = requests.session()
    resp = s.get(link, headers=headers, timeout=10)

    state_code, dist_code = re.findall('=(\d{1,2})', link)
    soup = BeautifulSoup(resp.content)
    court_codes = [x.attrs['value'].split("@")[1] for x in soup.find(id='court_complex_code').find_all('option')[1:]]
    print(court_codes)
    
    for court in court_codes:
        ActNodata = {
            'court_codeArr': court,
            'state_code': state_code,
            'dist_code': dist_code,
            'search_act': '',
            'action_code': 'fillActType',

        }
        response = s.post(
            'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
            headers=headers,
            data=ActNodata,
            timeout=5
        )
        codes = str(response.content).split("#")
        actCode = 0
        for code in codes:
            if "ipc" in code.lower() or "indian penal" in code.lower():
                actCode = code.split("~")[0]
                print(actCode, code)

        for case_type in ['Disposed']:

            solver = TwoCaptcha('3d5411601bd36ac99dab2eeade90f405')
            for x in range(3):
                try:
                    print(f"Trying {court} and {case_type}")
                    captcha_link = "https://services.ecourts.gov.in"+soup.find(id='captcha_image').attrs['src']
                    with open('image.png', 'wb') as f:
                        f.write(s.get(captcha_link, headers=headers, timeout=10).content)
                    captcha = solver.normal('image.png')
                    data = {
                        'court_codeArr': court,
                        'state_code': state_code,
                        'dist_code': dist_code,
                        'search_act': '',
                        'actcode': actCode,
                        'f': case_type,
                        'under_sec': section,
                        'action_code': 'showRecords',
                        'captcha': captcha['code'],
                        'lang': '',
                    }
                    print(f"Data is {data}")
                    response = s.post(
                        'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
                        headers=headers,
                        data=data,
                        timeout=5
                    )
                    
                    data = json.loads(response.content)
                    if data['con']=='Invalid Captcha':
                        "INVALID"
                        continue
                    else: 
                        break
                except Exception as e:
                    print(e)
                    continue

            data['stateCode'] = state_code
            data['distCode'] = dist_code
            with open(f"saloni/case_list/{section}_{court}_{state_code}_{dist_code}_{case_type}.txt", 'w') as f:
                f.write(json.dumps(data))


In [6]:
for section in ['302', '376']:
    for link in links:
        try:
            get_court_complex_cases(link, section=section)
        except requests.Timeout:
            print("TIMEOUT")


Starting  https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=4
['1,2,3,4']
12 12~Indian Penal Code
Trying 1,2,3,4 and Disposed
Data is {'court_codeArr': '1,2,3,4', 'state_code': '26', 'dist_code': '4', 'search_act': '', 'actcode': '12', 'f': 'Disposed', 'under_sec': '302', 'action_code': 'showRecords', 'captcha': 'lufz6c', 'lang': ''}
Starting  https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=5
['1,2,3,4']
12 12~Indian Penal Code
Trying 1,2,3,4 and Disposed
Data is {'court_codeArr': '1,2,3,4', 'state_code': '26', 'dist_code': '5', 'search_act': '', 'actcode': '12', 'f': 'Disposed', 'under_sec': '302', 'action_code': 'showRecords', 'captcha': '5l2fs7', 'lang': ''}
Starting  https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=2
['1,2,3,4']
12 12~Indian Penal Code
Trying 1,2,3,4 and Disposed
Data is {'court_codeArr': '1,2,3,4', 

In [3]:
links = """https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=4
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=5
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=2
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=1
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=9
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=8
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=3
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=7
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=6
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=11
https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=10"""
links = links.split("\n")
links

['https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=4',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=5',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=2',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=1',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=9',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=8',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=3',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=7',
 'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=6',
 

In [ ]:
soup = BeautifulSoup(resp.content)
[x.attrs['value'].split("@")[1] for x in soup.find(id='court_complex_code').find_all('option') if len(x)>1]

In [11]:
data = {
    'court_codeArr': '17',
    'state_code': '1',
    'dist_code': '21',
    'search_act': '',
    'actcode': '1',
    'f': 'Pending',
    'under_sec': '379',
    'action_code': 'fillActType',

}

response = s.post(
    'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
    headers=headers,
    data=data,
)
codes = str(response.content).split("#")
for code in codes:
    if "ipc" in code.lower() or "indian penal" in code.lower():
        print(code.split("~")[0])

1


In [19]:
import requests

cookies = {
    'PHPSESSID': 'k67skmheqfhh4gvvt34unu4h91',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'PHPSESSID=k67skmheqfhh4gvvt34unu4h91',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

response = requests.get(
    'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/display_pdf.php?filename=/orders/2016/202304411782016_31.pdf&caseno=SC/441178/2016&cCode=1&appFlag=&normal_v=1',
    cookies=cookies,
    headers=headers,
)
response.headers['Content-Type']

'application/pdf'